### Урок 2. #Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)#

### Домашнее задание
1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

[переход к ДЗ](#label_of_your_choice) 

<a id='## Домашнее задание'></a>
<!--ссылка в файле experiment00.ipynb-->

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import re

from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from razdel import tokenize
import pymorphy2

from gensim.models import LdaModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix

import warnings
warnings.simplefilter('ignore')

загрузим списки: новостей и пользователей.

In [2]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")
target = pd.read_csv("users_churn.csv")
print(news.shape)
print(users.shape)
print(target.shape)

(27000, 2)
(8000, 2)
(8000, 2)


Наши новости , пользователи и списки последних прочитанных новостей.

In [3]:
display(news.head(3), users.head(3), target.head(3))

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [4]:
#из gensim.test.utils импортировать common_texts
from gensim.corpora.dictionary import Dictionary

#Библиотека gensim.corpora.dictionary в Python является частью фреймворка gensim, который используется для работы с корпусами текстов, тематическим моделированием и векторными представлениями слов.
#Модуль Dictionary этой библиотеки предназначен для создания словаря из набора текстовых документов. Словарь содержит уникальные слова из текстов и их идентификаторы.
#Таким образом, этот модуль может использоваться для создания словарей, которые затем могут быть использованы для построения моделей тематического моделирования, поиска похожих документов и классификации текстов.
#Импортируя модуль Dictionary из библиотеки gensim.corpora.dictionary, можно использовать его функционал для создания словаря из текстовых документов и дальнейшей работы с ним в соответствии с поставленными задачами.

In [5]:
import numpy as np
#предобработка текстов
import re
#Библиотека re в Python предоставляет возможность работы с регулярными выражениями. Она используется для работы с текстовыми строками и позволяет осуществлять поиск, замену и манипуляции с текстом на основе определенных шаблонов.
#Импортируя эту библиотеку с помощью команды import re, вы можете использовать ее функционал для выполнения различных операций с текстом, например, для:
#Поиска подстроки в тексте
#Замены определенных символов или последовательностей символов в тексте
#Разбиения текста на подстроки по определенному разделителю
#Извлечения определенных значений из текста с помощью регулярных выражений
#Проверки соответствия текста определенному шаблону
#Кроме того, библиотека re широко используется в области обработки естественного языка (Natural Language Processing - NLP), например, для предобработки текста перед его анализом и классификацией.

from nltk.corpus import stopwords # так почему то не сработало
#Библиотека nltk (Natural Language Toolkit) в Python предоставляет инструменты для обработки естественного языка (Natural Language Processing - NLP).
#Она используется для работы с текстовыми данными и включает в себя различные модули.
#Модуль stopwords этой библиотеки содержит список стоп-слов для различных языков, которые обычно не несут важной смысловой нагрузки в тексте и могут быть исключены из анализа.
#Это позволяет снизить размерность данных и повысить качество анализа.
#Импортируя модуль stopwords из библиотеки nltk.corpus, вы можете использовать его функционал для удаления стоп-слов из текста перед его анализом или классификацией.
#Для этого можно использовать методы этого модуля, такие как words() или fileids(), чтобы получить список стоп-слов для определенного языка, а затем удалить их из текста с помощью простых операций в Python.
#import nltk
#nltk.download()
#Если библиотека nltk была установлена, но все равно возникла ошибка, то проблема может быть связана с отсутствием данных, которые необходимы для работы библиотеки.
#В этом случае необходимо выполнить дополнительные шаги, чтобы загрузить необходимые данные, например, загрузить корпусы текстов с помощью команды nltk.download().


from razdel import tokenize #!pip install razdel
#https://github.com/natasha/razdel
#Библиотека razdel в Python является инструментом для токенизации русскоязычных текстов.
#Она предоставляет функционал для разбиения текста на отдельные слова и пунктуационные знаки - токены.
#Модуль tokenize этой библиотеки содержит функцию tokenize(), которая принимает на вход текст и возвращает список токенов.
#Токенизация - это процесс разбиения текста на отдельные слова и знаки препинания, который является первым шагом в обработке текста перед его анализом или классификацией.
#Импортируя функцию tokenize() из библиотеки razdel, вы можете использовать ее для токенизации русскоязычных текстов на отдельные слова и пунктуационные знаки.
#Результатом работы этой функции будет список объектов Token, каждый из которых содержит информацию о токене, такую как его текст, начальную и конечную позиции в исходном тексте.
#Таким образом, этот модуль может быть использован для дальнейшей обработки текстовых данных, например, для извлечения ключевых слов или построения векторных представлений слов.

import pymorphy2  # pip install pymorphy2
#Библиотека pymorphy2 в Python является инструментом для морфологического анализа русскоязычных слов.
#Она предоставляет возможность определения частей речи, склонения и спряжения слов, а также получения их нормальных форм.
#Импортируя эту библиотеку с помощью команды import pymorphy2, вы можете использовать ее функционал для анализа и обработки русскоязычных слов.
#Для этого необходимо создать объект класса MorphAnalyzer(), который является основным инструментом библиотеки pymorphy2.
#Затем, вызывая различные методы этого объекта, можно производить морфологический анализ слов, например:
#Получать нормальную форму слова
#Определять его часть речи
#Склонять и спрягать слово в нужной форме
#Определять род, число и падеж слова
#Получать различные грамматические характеристики слова
#Этот модуль может быть использован для дальнейшей обработки текстовых данных, например, для лемматизации текста перед его анализом или классификацией,
#а также для создания словарей и морфологических анализаторов для русского языка.

In [6]:
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)
#Этот код открывает файл 'stopwords.txt' в режиме чтения с помощью оператора with и связывает его с переменной f.
#Затем он считывает содержимое файла, преобразует каждую строку в список слов, используя метод строк strip()
#для удаления лишних пробелов в начале и конце строки,и добавляет каждое слово в список additional_stopwords.
#Далее, код добавляет список additional_stopwords к списку stopword_ru, который является списком стоп-слов на русском языке.
#Это позволяет расширить список стоп-слов, используемый в некотором приложении или алгоритме, за счет добавления дополнительных слов, указанных в файле 'stopwords.txt'.

776

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [9]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: total: 27.6 s
Wall time: 27.6 s


In [10]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)
#В этом коде цифра 1 указывает на ось, по которой функция apply() будет применяться к столбцу 'title' объекта DataFrame news.
#Параметр axis метода apply() по умолчанию равен 0 и означает, что функция должна применяться к каждому столбцу,
#но в данном случае он задан явно и равен 1, что означает, что функция должна применяться к каждой строке столбца 'title'.
#Таким образом, функция clean_text() будет вызвана для каждого элемента столбца 'title' в DataFrame news и результат 
#ее выполнения будет сохранен в этом же столбце, заменяя исходные значения.

CPU times: total: 3min 30s
Wall time: 3min 31s


А теперь в 3 строчки обучим нашу модель

In [12]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]
# Создать корпус из списка текстов
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [13]:
common_dictionary[100]

'выбросить'

Все просто - это словарь наших слов

Запускаем обучение

In [14]:
from gensim.models import LdaModel
# Обучим модель на корпусе.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)
#Данный код использует библиотеку Gensim и класс LdaModel для обучения модели LDA (Latent Dirichlet Allocation) на заданном корпусе текстовых документов common_corpus.
#LDA является вероятностной моделью тематического моделирования, которая позволяет выделить скрытые темы, представленные распределением слов в документах.
#Она основана на предположении, что каждый документ в корпусе можно представить как смесь нескольких тем с различными вероятностями,
#а каждая тема может быть представлена распределением слов.
#В параметрах модели указывается количество тем num_topics, которые модель должна выделить, а также словарь id2word,
#содержащий отображение между идентификаторами слов и их текстовыми представлениями.
#При необходимости можно указать также дополнительные параметры, такие как количество проходов passes по корпусу.
#После обучения модели LDA на заданном корпусе можно использовать ее для выделения тем в новых текстовых документах,
#а также для анализа и интерпретации результатов, таких как топ-слова каждой темы или вероятности принадлежности документа к каждой теме.

In [15]:
from gensim.test.utils import datapath
# Сохранить модель на диск.
temp_file = datapath("model.lda")
lda.save(temp_file)
# Загрузите потенциально предварительно обученную модель с диска.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [16]:
# Создайте новый корпус из ранее невиданных документов.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(5, 0.087219715),
 (6, 0.06633881),
 (8, 0.429779),
 (12, 0.09568536),
 (20, 0.21605226),
 (21, 0.08776029)]

In [17]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]
#Ниже код печатает только слова 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: эксперимент налог открыться платёжный сибирский понятие дальневосточный
topic_1: газ турция турецкий приток улица аудитория метан
topic_2: президент государство который это глава путин свой
topic_3: высота восток золото искусство музыка поражать присудить
topic_4: оборудование предмет отель ндс индонезия водитель колебаться
topic_5: обнаружить человек тело взрыв произойти место город
topic_6: год санкция который nn закон это китай
topic_7: участок торговый фильм площадь документация скот выставочный
topic_8: это который жизнь свой первый весь очень
topic_9: проверка депутат товар который россия совет сообщество
topic_10: пенсия возраст школа километр год страдать египет
topic_11: год это который россия дело банк также
topic_12: это который мочь всё весь свой говорить
topic_13: исследование который nn данные мозг мочь источник
topic_14: мальчик челябинский приступ пляж timesn выписка власов
topic_15: ребёнок достигать британский подросток больной лётчик величина
topic_16: компа

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [18]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [20]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.066326,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.033079,0.0
1,4896,0.000000,0.0,0.099155,0.0,0.0,0.462696,0.058424,0.0,0.199080,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.160612,0.000000,0.000000,0.0
2,4897,0.000000,0.0,0.000000,0.0,0.0,0.087215,0.066407,0.0,0.429798,...,0.0,0.0,0.0,0.0,0.0,0.216053,0.087759,0.000000,0.000000,0.0
3,4898,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.669195,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.040152,0.000000,0.000000,0.0
4,4899,0.250821,0.0,0.290592,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.103548,0.000000,0.0


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы\
Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [21]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [22]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [23]:
doc_dict[293622]

array([0.03408372, 0.01808866, 0.        , 0.        , 0.        ,
       0.        , 0.1191908 , 0.        , 0.13824035, 0.        ,
       0.        , 0.17380114, 0.        , 0.08436631, 0.        ,
       0.        , 0.02537937, 0.20842977, 0.07661167, 0.0371021 ,
       0.02634536, 0.        , 0.        , 0.05045659, 0.        ])

<a id='label_of_your_choice'></a>

### Домашнее задание

#### 1.Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)


TF-IDF (term frequency-inverse document frequency (частота термина, обратная частоте документа)) – это статистический показатель, используемый в информационном поиске и анализе текстовых данных. Он позволяет оценить важность слова в контексте документа или коллекции документов.\
TF-IDF вычисляется на основе двух показателей:\
TF (term frequency) – отношение числа вхождений слова к общему числу слов в документе.\
IDF (inverse document frequency) – логарифм отношения общего числа документов к числу документов, содержащих данное слово.\
Таким образом, TF-IDF для слова w в документе d рассчитывается по формуле:
TF-IDF(w, d) = TF(w, d) * IDF(w)

Векторизация текста на основе TF-IDF позволяет представить документы в виде набора чисел, каждое из которых представляет собой вес соответствующего слова в документе.\
Эти веса позволяют отранжировать документы по степени их сходства с запросом, что используется, например, в системах информационного поиска.

В Python для вычисления TF-IDF можно использовать класс TfidfVectorizer из библиотеки scikit-learn.\
Этот класс позволяет вычислить матрицу TF-IDF для заданного набора текстовых документов.

In [ ]:
#### 2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
# вариант из лекции
#user_articles_list = users['articles'].iloc[33]

#def get_user_embedding(user_articles_list):
#    user_articles_list = eval(user_articles_list)
#    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#    user_vector = np.mean(user_vector, 0)  # вариант из лекции - среднее
#    return user_vector 


# вариант из лекции - среднеарифметическое
#def get_user_embedding_mean(user_articles_list):
#    user_articles_list = eval(user_articles_list)
#    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#    user_vector = np.mean(user_vector, 0)
#    return user_vector
# вариант  - среднее
#def get_user_embedding_median(user_articles_list):
#    user_articles_list = eval(user_articles_list)
#    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#    user_vector = np.median(user_vector, 0)
#    return user_vector
# вариант  - максимальное
#def get_user_embedding_max(user_articles_list):
#    user_articles_list = eval(user_articles_list)
#    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
#    user_vector = np.max(user_vector, 0)
#    return user_vector

In [24]:
def get_user_embedding(user_articles_list, metric=np.mean):
    
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    
    return user_vector

In [25]:
def get_metrics(metric): 

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                        X['churn'], random_state=0)

    logreg = LogisticRegression()

    logreg.fit(X_train, y_train)

    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)

    ix = np.argmax(fscore)

    return roc_auc, precision[ix], recall[ix], fscore[ix]

#### 3. Повторить п.2, но используя уже не медиану, а max


#### 5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [27]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
stats = pd.DataFrame(columns=metrics.keys())

for k, v in metrics.items():
    roc_auc, precision, recall, fscore = get_metrics(v)
    stats.loc[f'roc_auc', k] = roc_auc
    stats.loc[f'precision', k] = precision
    stats.loc[f'recall', k] = recall
    stats.loc[f'fscore', k] = fscore
    
stats

,mean,median,max
roc_auc,0.941745,0.972219,0.974343
precision,0.614094,0.768924,0.771084
recall,0.746939,0.787755,0.783673
fscore,0.674033,0.778226,0.777328


#### 6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных


С увеличием порога вхождения все метрики качества стремительно растут.

#### далее продолжение кода по лекции.

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

In [29]:
users['articles'].iloc[33]

'[323329, 321961, 324743, 323186, 324632, 474690]'

In [30]:
" ".join(news[news['doc_id']==323186]['title'].iloc[0])

'глава российский мид сергей лавров опровергнуть появиться сми информация якобы готовиться обмен декларация россия сша сотрудничество сфера сообщать риа новость nn читать сообщение разговаривать автор сообщение знать откуда автор источник какихлибо основание подобный род репортаж знать откуда информация появиться сказать журналист итог встреча госсекретарь сша джон керри nn позиция свой изложить декларация напринимать достаточно рамка обсе рамка совет россия нато высокий уровень продекларировать всё обеспечивать неделимость безопасность никто обеспечивать свой безопасность счёт безопасность продолжить министр слово лавров москва считать система нато создавать проблема наш безопасность поэтому декларация недостаточно мочь договариваться совместный система россия предлагать ещё начинать год президент путин посещать сша нужно вести речь очередной декларация гарантия который проверять объективный военнотехнический критерий гарантия ненаправленность система против российский ядерный потенци

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [31]:
user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x), 1)])
user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings['uid'] = users['uid'].values
user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings.head(5)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.009421,0.014228,0.040398,0.000000,0.003546,0.074359,0.035124,0.000000,0.033429,...,0.016315,0.004230,0.117030,0.036694,0.052311,0.179399,0.000000,0.021801,0.008409,0.000000
1,u108690,0.001880,0.000000,0.093631,0.000000,0.000000,0.021277,0.046849,0.002868,0.028899,...,0.004806,0.003431,0.048189,0.013626,0.026829,0.075029,0.000000,0.014378,0.012813,0.000000
2,u108339,0.002053,0.005642,0.101173,0.000000,0.000000,0.093521,0.058707,0.000000,0.022930,...,0.000000,0.020143,0.061343,0.013836,0.062732,0.080471,0.000000,0.019639,0.000000,0.000000
3,u101138,0.003259,0.000000,0.040212,0.005082,0.000000,0.031610,0.003941,0.000000,0.326751,...,0.004481,0.020979,0.014311,0.114287,0.007393,0.047658,0.018708,0.010888,0.099998,0.007269
4,u108248,0.000000,0.000000,0.069695,0.000000,0.000000,0.016852,0.024116,0.000000,0.059302,...,0.000000,0.006026,0.074648,0.012014,0.022132,0.066881,0.000000,0.022019,0.002040,0.000000


Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [ ]:
target = pd.read_csv("users_churn.csv")
target.head(3)

In [ ]:
X = pd.merge(user_embeddings, target, 'left')
X.head(3)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)

In [ ]:
logreg = LogisticRegression()
#обучим наш пайплайн
logreg.fit(X_train, y_train)

In [ ]:
#наши прогнозы для тестовой выборки
preds = logreg.predict_proba(X_test)[:, 1]
preds[:10]

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

### Рассчитаем Precision, Recall, F_score

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

In [ ]:
roc_auc_score(y_test, preds)

В целом мы видим, что получившиеся векторные представления содержат какой-то сигнал и позволяют решать нашу прикладную задачу. 

#### Ссылки

1. http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf

Для определения значимости признаков в наборе данных существует несколько методов, в том числе и методы выбора признаков, основанные на их важности. Один из таких методов – это метод исключения признаков (feature selection), который заключается в том, чтобы выбрать подмножество признаков, наиболее важных для предсказания целевой переменной, и исключить остальные.

Один из подходов к выбору наиболее важных признаков – это использование моделей машинного обучения с возможностью оценки важности признаков. Например, для алгоритмов деревьев решений (decision trees) или случайного леса (random forest) можно вычислить важность каждого признака на основе его вклада в улучшение разбиения данных. Также существуют методы, основанные на регрессионных моделях, например, Lasso-регрессия (Least Absolute Shrinkage and Selection Operator), которая позволяет отбирать признаки, имеющие наибольший вклад в предсказание целевой переменной при ограниченной сложности модели.

Кроме того, для определения важности признаков можно использовать статистические методы, такие как анализ дисперсии (ANOVA) или корреляционный анализ, которые позволяют оценить вклад каждого признака в общую дисперсию целевой переменной или взаимосвязь между признаками.

Главное, что следует отметить, это то, что модели ARTM и BigARTM на данный момент являются одними из наиболее актуальных методов тематического моделирования текстов, позволяющих учитывать разнообразные факторы, такие как мультиязычность, специфические свойства документов и т.д. Они также имеют высокую скорость обработки и масштабируемость, что делает их привлекательными для использования в больших проектах.

Кроме того, книга описывает некоторые новейшие исследования, связанные с применением моделей ARTM и BigARTM в различных областях, таких как анализ социальных сетей, медицинские исследования и другие. В целом, можно сказать, что тематическое моделирование текстов с использованием моделей ARTM и BigARTM продолжает развиваться и находить новые применения в различных областях.

Книга "BigARTM: сравнение моделей тематического моделирования на больших корпусах" написана профессором В.В. Воронцовым и является обзором современных методов тематического моделирования, а также представляет новый метод под названием BigARTM.

Основные знания из этой книги можно свести к следующему:

Тематическое моделирование - это метод машинного обучения, который позволяет извлекать темы из больших наборов текстовых данных.

Существует несколько подходов к тематическому моделированию, включая Latent Dirichlet Allocation (LDA), Probabilistic Latent Semantic Analysis (PLSA), и тематические модели с разреженным векторным представлением (sparse topic models).

BigARTM - это новый метод тематического моделирования, который использует комбинацию различных моделей и позволяет работать с большими наборами данных.

Для эффективного применения тематического моделирования важно правильно выбирать параметры модели и проводить ее оценку, в том числе с помощью метрик качества.

Тематическое моделирование имеет широкий спектр приложений, включая анализ социальных сетей, обработку текстов на естественном языке, анализ клиентской обратной связи и многое другое.

В целом, книга представляет обзор основных методов и инструментов тематического моделирования и является полезным источником информации для исследователей и практиков, работающих в области анализа текстовых данных.


2. https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation



Скрытое распределение Дирихле - это вероятностное распределение, используемое в статистическом моделировании для описания распределения вероятностей наблюдаемых переменных, которые являются смесью неизвестных распределений.

В частности, скрытое распределение Дирихле может использоваться в задаче тематического моделирования, где необходимо определить скрытые темы, которые могут быть выявлены из набора текстовых документов. В этом случае каждый документ рассматривается как смесь различных тем, а каждая тема представляется скрытым распределением Дирихле.

Скрытое распределение Дирихле имеет несколько параметров, которые могут быть оценены с использованием различных методов оптимизации, таких как EM-алгоритм или вариационный метод. Эти методы позволяют определить параметры распределения на основе наблюдаемых данных.

Одно из применений скрытого распределения Дирихле - это модель LDA (Latent Dirichlet Allocation), которая используется для тематического моделирования. В этой модели каждый документ представляется как смесь нескольких скрытых тем, а каждая тема - как распределение Дирихле над словами.

Скрытое распределение Дирихле является важным инструментом в статистическом моделировании и используется во многих других областях, таких как машинное обучение, байесовская статистика и теория информации.


